In [ ]:
%pip install -U langgraph

In [3]:
from langgraph.func import entrypoint, task
from langgraph.types import interrupt, Command
from langgraph.checkpoint.memory import MemorySaver
import os

# from google.colab import userdata
# os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')

@task
def screen_resume(resume: str) -> dict:
    print("--- TASK: AI screening resume ---")
    skills = []
    for skill in ["Python", "Java", "Leadership"]:
        if skill.lower() in resume.lower():
            skills.append(skill)

    score = len(skills) * 30
    recommendation = "Interview" if score >= 60 else "Reject"

    print(f"    AI Recommendation: {recommendation} (Score: {score})")

    return {
        "skills": skills,
        "score": score,
        "ai_recommendation": recommendation
    }

@entrypoint(checkpointer=MemorySaver())
def hiring_workflow(resume: str) -> str:
    print("\n--- WORKFLOW: Starting hiring process ---")

    screening_result = screen_resume(resume)

    print("--- WORKFLOW: Pausing for human approval ---")
    human_decision = interrupt("Do you approve this candidate? (yes/no)")

    print(f"--- WORKFLOW: Resuming with human decision: '{human_decision}' ---")

    if human_decision == "yes":
        final_decision = f"HIRED - AI recommended '{screening_result['ai_recommendation']}'"
    else:
        final_decision = "REJECTED - Human override"

    print(f"--- WORKFLOW: Final decision is '{final_decision}' ---")
    return final_decision

if __name__ == "__main__":
    thread_config = {"thread_id": "hire_001"}
    resume_text = "Senior Python developer with Java and leadership skills"

    print("="*60)
    print("STEP 1: RUNNING UNTIL INTERRUPT")
    print("="*60)
    hiring_workflow.invoke(resume_text, config=thread_config)

    print("\n" + "="*60)
    print("STEP 2: RESUMING WITH HUMAN INPUT ('yes')")
    print("="*60)

    final_result = hiring_workflow.invoke(
        None,
        config=thread_config,
        resume=Command(resume="yes")
    )

    print(f"\n--- FINAL RESULT ---")
    print(final_result)

STEP 1: RUNNING UNTIL INTERRUPT

--- WORKFLOW: Starting hiring process ---
--- WORKFLOW: Pausing for human approval ---
--- TASK: AI screening resume ---
    AI Recommendation: Interview (Score: 90)

STEP 2: RESUMING WITH HUMAN INPUT ('yes')

--- WORKFLOW: Starting hiring process ---
--- WORKFLOW: Pausing for human approval ---

--- FINAL RESULT ---
{'__interrupt__': [Interrupt(value='Do you approve this candidate? (yes/no)', id='b25ac9acd55afa10eca5497e9bb0a99a')]}
